In [12]:
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy
import mysql.connector
import re

In [13]:
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(
    client_id='cf6c6ea8e17642d39e097d163531226d',
    client_secret='f7580b3d0d0f4a61b99aff3d4bc1c7f5'
))

In [14]:
db_config = {
    'host': 'localhost',
    'user': 'root',
    'password': 'root1234gsixK',
    'database': 'Spotify_database'
}

In [15]:
connection = mysql.connector.connect(**db_config)
cursor = connection.cursor()

In [16]:
file_path = "track_urls.txt"
with open(file_path, 'r') as file:
    track_urls = file.readlines()

In [17]:
for track_url in track_urls:
    track_url = track_url.strip()
    try:
        track_id = re.search(r'track/([a-zA-Z0-9]+)', track_url).group(1)
        track = sp.track(track_id)

        track_data = {
            'Track Name': track['name'],
            'Artist': track['artists'][0]['name'],
            'Album': track['album']['name'],
            'Popularity': track['popularity'],
            'Duration (minutes)': track['duration_ms'] / 60000
        }

        insert_query = """
        INSERT INTO spotify_tracks (track_name, artist, album, popularity, duration_minutes)
        VALUES (%s, %s, %s, %s, %s)
        """
        cursor.execute(insert_query, (
            track_data['Track Name'],
            track_data['Artist'],
            track_data['Album'],
            track_data['Popularity'],
            track_data['Duration (minutes)']
        ))
        connection.commit()

        print(f"Track '{track_data['Track Name']}' by {track_data['Artist']} inserted into the database.")

    except Exception as e:
        print(f"Error_processing URL: {track_url}, Error: {e}")


Track 'Aazhi Mazhai Kanna' by A.R. Rahman inserted into the database.
Track 'Aga Naga (From "Ponniyin Selvan Part-2")' by A.R. Rahman inserted into the database.
Track 'Chinnanjiru (Marumurai)' by A.R. Rahman inserted into the database.
Track 'Chinnanjiru Nilave' by A.R. Rahman inserted into the database.
Track 'Ilaiyor Soodaar' by A.R. Rahman inserted into the database.
Track 'Shivoham' by A.R. Rahman inserted into the database.
Track 'Veera Raja Veera' by Composition Based on a Dagarvani Tradition Dhrupad inserted into the database.
Track 'Alaikadal (From "Ponniyin Selvan Part - 1")' by A.R. Rahman inserted into the database.
Track 'Chola Chola (From "Ponniyin Selvan Part - 1")' by A.R. Rahman inserted into the database.
Track 'Devaralan Aattam (From "Ponniyin Selvan Part - 1")' by A.R. Rahman inserted into the database.
Track 'Ponni Nadhi (From "Ponniyin Selvan Part - 1")' by A.R. Rahman inserted into the database.
Track 'Ratchasa Maamaney (From "Ponniyin Selvan Part - 1")' by A.R. 

In [18]:
cursor.close()
connection.close()